# TimesFM 통신사 손익전망 분석

이 노트북은 통신사의 5G, LTE, 3G 데이터를 분석하여 손익전망을 예측합니다.

## 📋 목차
1. 환경 설정 및 라이브러리 import
2. 데이터 로드 및 검증
3. 데이터 전처리 및 탐색적 분석
4. 모델 훈련 및 예측
5. 결과 시각화 및 분석
6. 비즈니스 인사이트 도출

## 1. 환경 설정 및 라이브러리 import

## 2. 데이터 로드 및 검증

## 3. 데이터 전처리 및 탐색적 분석

## 4. 모델 훈련 및 예측

## 5. 결과 시각화 및 분석

## 6. 비즈니스 인사이트 도출

## 📝 결론

이 노트북을 통해 통신사의 기술별 사용자 수와 수익성을 예측하고, 전략적 인사이트를 도출했습니다.

### 주요 결과:
- 📊 기술별 사용자 수 예측
- 💰 수익성 전망 분석
- 🎯 전략적 권장사항 도출

### 다음 단계:
1. 📁 `results/` 폴더에서 상세한 결과 파일 확인
2. 📊 대화형 대시보드 (`interactive_dashboard.html`) 활용
3. 📈 정기적인 모델 업데이트 및 재훈련
4. 🎯 비즈니스 전략에 예측 결과 반영

---
**참고**: 더 자세한 사용법은 `USER_GUIDE.md` 파일을 참고하세요.

In [ ]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 프로젝트 모듈 import
import sys
sys.path.append('../src')

from data_processing import TelecomDataProcessor
from forecasting import TelecomForecaster

# 한글 폰트 설정 (한글 출력 시)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ 라이브러리 import 완료")

In [ ]:
# 데이터 로더 생성
processor = TelecomDataProcessor()

# 데이터 파일 경로 설정 (실제 파일명으로 변경하세요)
file_path = '../data/raw/your_telecom_data.xlsx'  # 여기에 실제 파일명 입력

print(f"📁 데이터 파일 경로: {file_path}")

# 파일 존재 확인
import os
if os.path.exists(file_path):
    print("✅ 파일 존재 확인")
else:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
    print("📝 data/raw/ 폴더에 엑셀 파일을 넣고 경로를 수정하세요.")

In [ ]:
# 데이터 로드
try:
    df = processor.load_data(file_path)
    print("✅ 데이터 로드 완료")
    print(f"📊 데이터 크기: {df.shape}")
    print(f"📅 기간: {df['date'].min()} ~ {df['date'].max()}")
except Exception as e:
    print(f"❌ 데이터 로드 실패: {e}")
    print("📝 파일 경로와 형식을 확인하세요.")

In [ ]:
# 데이터 검증
if 'df' in locals():
    is_valid = processor.validate_data(df)
    if is_valid:
        print("✅ 데이터 검증 통과")
        
        # 데이터 미리보기
        print("\n📋 데이터 미리보기:")
        display(df.head())
        
        print("\n📊 데이터 정보:")
        print(df.info())
        
        print("\n🔍 결측치 확인:")
        print(df.isnull().sum())
    else:
        print("❌ 데이터 검증 실패")
        print("📝 USER_GUIDE.md 파일을 참고하여 데이터 형식을 확인하세요.")

In [ ]:
# 데이터 정리 및 전처리
if 'df' in locals() and processor.validate_data(df):
    print("🔄 데이터 전처리 시작...")
    
    # 데이터 정리
    df_clean = processor.clean_data(df)
    print("✅ 데이터 정리 완료")
    
    # 특성 생성
    df_features = processor.create_features(df_clean)
    print("✅ 특성 생성 완료")
    
    print(f"📊 전처리된 데이터 크기: {df_features.shape}")
    print("\n📋 전처리된 데이터 미리보기:")
    display(df_features.head())

In [ ]:
# 기술별 사용자 수 추이 시각화
if 'df_features' in locals():
    plt.figure(figsize=(14, 8))
    
    plt.subplot(2, 2, 1)
    plt.plot(df_features['date'], df_features['5g_users'], label='5G', linewidth=2)
    plt.plot(df_features['date'], df_features['lte_users'], label='LTE', linewidth=2)
    plt.plot(df_features['date'], df_features['3g_users'], label='3G', linewidth=2)
    plt.title('기술별 사용자 수 추이', fontsize=14, fontweight='bold')
    plt.xlabel('날짜')
    plt.ylabel('사용자 수')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(2, 2, 2)
    plt.plot(df_features['date'], df_features['5g_revenue'], label='5G', linewidth=2)
    plt.plot(df_features['date'], df_features['lte_revenue'], label='LTE', linewidth=2)
    plt.plot(df_features['date'], df_features['3g_revenue'], label='3G', linewidth=2)
    plt.title('기술별 매출 추이', fontsize=14, fontweight='bold')
    plt.xlabel('날짜')
    plt.ylabel('매출')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(2, 2, 3)
    plt.plot(df_features['date'], df_features['profit'], label='총 이익', linewidth=2, color='green')
    plt.title('총 이익 추이', fontsize=14, fontweight='bold')
    plt.xlabel('날짜')
    plt.ylabel('이익')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(2, 2, 4)
    plt.plot(df_features['date'], df_features['profit_margin'], label='이익률', linewidth=2, color='red')
    plt.title('이익률 추이', fontsize=14, fontweight='bold')
    plt.xlabel('날짜')
    plt.ylabel('이익률 (%)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 예측기 생성 및 설정
if 'df_features' in locals():
    print("🤖 모델 훈련 시작...")
    
    # 예측기 생성
    forecaster = TelecomForecaster()
    
    # 예측 설정
    target_columns = ['5g_users', 'lte_users', '3g_users']
    forecast_steps = 12  # 12개월 예측
    
    print(f"🎯 예측 대상: {target_columns}")
    print(f"📅 예측 기간: {forecast_steps}개월")
    
    # GPU 사용 가능 여부 확인
    import torch
    if torch.cuda.is_available():
        print("🚀 GPU 사용 가능 - 빠른 훈련 예상")
    else:
        print("💻 CPU 사용 - 훈련 시간이 오래 걸릴 수 있습니다")

In [ ]:
# 전체 파이프라인 실행
if 'forecaster' in locals() and 'file_path' in locals():
    try:
        print("🔄 예측 파이프라인 실행 중...")
        print("⏰ 시간이 오래 걸릴 수 있습니다. 잠시 기다려주세요...")
        
        results = forecaster.run_full_pipeline(
            file_path=file_path,
            target_columns=target_columns,
            forecast_steps=forecast_steps
        )
        
        print("✅ 예측 완료!")
        print(f"📁 결과 파일이 results/ 폴더에 저장되었습니다.")
        
    except Exception as e:
        print(f"❌ 예측 실패: {e}")
        print("📝 데이터 형식과 파일 경로를 다시 확인하세요.")

In [ ]:
# 예측 결과 확인
if 'results' in locals():
    print("📊 예측 결과 분석")
    
    # 예측 결과 미리보기
    if 'forecast' in results:
        print("\n🔮 예측 결과 미리보기:")
        display(results['forecast'].head())
        
        print(f"\n📅 예측 기간: {results['forecast'].index[0]} ~ {results['forecast'].index[-1]}")
        
    # 수익성 분석 결과
    if 'profitability' in results:
        print("\n💰 수익성 분석 결과:")
        display(results['profitability'].head())

In [ ]:
# 예측 결과 시각화
if 'results' in locals() and 'forecast' in results:
    # 기술별 사용자 수 예측
    plt.figure(figsize=(15, 10))
    
    # 과거 데이터와 예측 데이터 결합
    historical = df_features[['date'] + target_columns].set_index('date')
    forecast = results['forecast'][target_columns]
    
    # 시각화
    for i, col in enumerate(target_columns, 1):
        plt.subplot(3, 1, i)
        
        # 과거 데이터
        plt.plot(historical.index, historical[col], label='과거 데이터', linewidth=2)
        
        # 예측 데이터
        plt.plot(forecast.index, forecast[col], label='예측 데이터', linewidth=2, linestyle='--', color='red')
        
        plt.title(f'{col.upper()} 사용자 수 예측', fontsize=14, fontweight='bold')
        plt.xlabel('날짜')
        plt.ylabel('사용자 수')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 예측 결과 분석 및 인사이트
if 'results' in locals() and 'forecast' in results:
    print("💡 비즈니스 인사이트 분석")
    print("=" * 50)
    
    # 기술별 성장률 분석
    latest_actual = df_features.iloc[-1]
    latest_forecast = results['forecast'].iloc[-1]
    
    for tech in ['5g', 'lte', '3g']:
        actual_users = latest_actual[f'{tech}_users']
        forecast_users = latest_forecast[f'{tech}_users']
        growth_rate = ((forecast_users - actual_users) / actual_users) * 100
        
        print(f"\n📱 {tech.upper()} 기술:")
        print(f"   현재 사용자: {actual_users:,.0f}명")
        print(f"   12개월 후 예상: {forecast_users:,.0f}명")
        print(f"   예상 성장률: {growth_rate:+.1f}%")
        
        if growth_rate > 10:
            print(f"   💚 강한 성장 예상 - 투자 확대 권장")
        elif growth_rate > 0:
            print(f"   📈 안정적 성장 예상 - 유지 관리")
        else:
            print(f"   ⚠️ 감소 예상 - 전략 재검토 필요")

In [ ]:
# 전략적 권장사항
if 'results' in locals():
    print("\n🎯 전략적 권장사항")
    print("=" * 50)
    
    print("\n📡 5G 전략:")
    print("   ✅ 5G 투자 확대 및 마케팅 강화")
    print("   ✅ 신규 서비스 개발 및 고객 유치")
    
    print("\n📶 LTE 전략:")
    print("   📊 LTE 안정적 운영 유지")
    print("   📊 비용 최적화 및 효율성 개선")
    
    print("\n📱 3G 전략:")
    print("   🔄 3G 서비스 점진적 축소 계획")
    print("   🔄 사용자 이전 프로그램 강화")
    
    print("\n💰 수익성 개선 방안:")
    print("   📈 ARPU 향상을 위한 부가서비스 개발")
    print("   📈 고객 충성도 프로그램 강화")
    print("   📈 네트워크 효율성 개선으로 비용 절감")
    print("   📈 신규 수익 모델 발굴")